# Fake News Detector

## Installing Necessary Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Loading the data

In [ ]:
data_fake=pd.read_csv('Datasets/Fake.csv')
data_true=pd.read_csv('Datasets/True.csv')

### Data Preview 

In [ ]:
data_fake.head()

In [ ]:
data_true.tail()

In [ ]:
data_fake["class"]=0
data_true['class']=1

In [ ]:
data_fake.shape, data_true.shape

In [ ]:
# removing the last 10 rows from both datasets to use them for manual testing later.

data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i],axis = 0, inplace = True)

    
data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i],axis = 0, inplace = True)

In [ ]:
data_fake_manual_testing['class']=0
data_true_manual_testing['class']=1

In [ ]:
data_fake_manual_testing.head(10)

In [ ]:
data_true_manual_testing.head(10)

In [ ]:
data_merge=pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

In [ ]:
data=data_merge.drop(['title','subject','date'], axis = 1)

In [ ]:
data.isnull().sum() 

In [ ]:
data = data.sample(frac = 1)

In [ ]:
data.head()

In [ ]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [ ]:
data.columns

In [ ]:
data.head()

## Preprocessing Text

#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]','',text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+','',text)
    text = re.sub('<.*?>+',b'',text)
    text = re.sub('[%s]' % re.escape(string.punctuation),'',text)
    text = re.sub('\w*\d\w*','',text)
    return text

In [ ]:
data['text'] = data['text'].apply(wordopt)

#### Defining dependent and independent variable as x and y

In [ ]:
x = data['text']
y = data['class']

## Training the model

#### Splitting the dataset into training set and testing set. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.20)

### Extracting Features from the Text

#### Convert text to vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

In [ ]:
pred_lr = LR.predict(xv_test)

In [ ]:
pred_lr

In [ ]:
LR.score(xv_test, y_test)

In [ ]:
print (classification_report(y_test, pred_lr))

## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

In [ ]:
print (classification_report(y_test, pred_dt))

## Gradient Boost Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)


In [ ]:
pred_gb = GB.predict(xv_test)

In [ ]:
GB.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_gb))

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

In [ ]:
pred_rf = RF.predict(xv_test)

In [ ]:
RF.score(xv_test, y_test)

In [ ]:
print (classification_report(y_test, pred_rf))

## Testing the Model

In [ ]:
def output_lable(n):
    if n==0:
        return "FAKE News"
    elif n==1:
        return "REAL News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    
    return print("\n\nLR Predicition: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction:{}".format(output_lable(pred_LR[0]),
                                                                                                             output_lable(pred_DT[0]),
                                                                                                             output_lable(pred_GB[0]),
                                                                                                             output_lable(pred_RF[0])))

In [ ]:
news = str(input()) 
manual_testing(news)